In [1]:
-- scaling macro because we can't have constants in sql
CREATE OR REPLACE MACRO per_county(multiple) AS multiple * 500;

,Count


In [1]:
-- Create a countries table
-- https://www.gov.uk/government/publications/open-standards-for-government/country-codes
CREATE OR REPLACE TABLE countries AS (
SELECT 
    UNNEST(range(1,5)) as id, 
    UNNEST(['GB-ENG', 'GB-NIR','GB-SCT', 'GB-WAL']) as country
);
SELECT * FROM countries;

,id,country
0,1,GB-ENG
1,2,GB-NIR
2,3,GB-SCT
3,4,GB-WAL


In [2]:
-- Create counties table
CREATE OR REPLACE TABLE counties AS (
    SELECT 
        range+1 as id, 
        oversample('countries', 'id') as country_id, 
        mimesis_address_city() as county_name, 
        '' as prefix 
FROM range(per_county(2)));

-- maybe add pre, post changes like east, west, -shire etc.
-- UPDATE counties SET prefix = ucase(concat(substr(name, 1,1), substr(name, 3,1)));
UPDATE counties SET prefix = substr(county_name, 1,2);
UPDATE counties SET prefix = concat(substr(string_split(county_name, ' ')[1], 1, 1), substr(string_split(county_name, ' ')[2], 1, 1)) WHERE county_name LIKE '% %';
UPDATE counties SET prefix = concat(substr(county_name, 1,1), substr(county_name, -1, 1)) WHERE prefix in (SELECT prefix FROM counties GROUP BY prefix HAVING (COUNT(*) > 1));
-- TODO: Update further duplicates from a window function that makes it look like S1A, S2A, S3A ...

UPDATE  counties SET prefix = ucase(prefix);

-- drop duplicated prefixes
DELETE FROM counties WHERE prefix IN (
    SELECT prefix
    FROM counties
    GROUP BY prefix
    HAVING (COUNT(*) > 1));

SELECT count(*) FROM counties;

,count_star()
0,103


In [4]:
CREATE OR REPLACE SEQUENCE town_seq START 1;
CREATE OR REPLACE TEMP TABLE tmp_county_postcodes AS (SELECT range+1 as id, oversample('counties', 'id') as county_id from range(per_county(10)));
CREATE OR REPLACE TABLE county_postcodes AS (SELECT id, county_id, row_number() OVER (PARTITION BY county_id) as division FROM tmp_county_postcodes);

CREATE OR REPLACE TABLE towns as (
    SELECT
        nextval('town_seq') as id,
        concat(prefix, division) as outcode, 
        faker_city() as town_name,
        county_id
    FROM county_postcodes cpo 
    JOIN counties ON cpo.county_id = counties.id 
    JOIN countries on counties.country_id = countries.id);

select count(*) from towns;

,count_star()
0,5000


In [5]:
CREATE OR REPLACE SEQUENCE incode_seq START 1;
CREATE OR REPLACE TEMP TABLE tmp_incodes AS (
    SELECT oversample('towns', 'id') as town_id FROM range(per_county(200)) ORDER BY town_id
);

CREATE OR REPLACE TABLE incodes_sector AS (
    SELECT 
        nextval('incode_seq') as id,
        town_id, 
        row_number() OVER (PARTITION BY town_id) as incode_division 
    FROM incodes
);

CREATE OR REPLACE TABLE incodes_street AS (
    SELECT
        range as id,
        concat(mimesis_address_street_name(), ' ', faker_street_suffix()) as street_name,
        oversample('incodes_sector', 'id') as sector_id,
        xeger('A[A-Z]{1}') as incode_street
    FROM range(per_county(1000))
);

CREATE OR REPLACE TABLE streets AS (
    SELECT ist.id, town_id, concat(incode_division, incode_street) as incode, street_name FROM incodes_sector ise JOIN incodes_street as ist ON ist.sector_id = ise.id
);

SELECT * FROM streets ORDER BY town_id, incode LIMIT 10;

DROP TABLE IF EXISTS incodes_sector;
DROP TABLE IF EXISTS incodes_street;


,Success


In [6]:
CREATE OR REPLACE SEQUENCE address_seq START 1;
CREATE OR REPLACE TEMP TABLE tmp_addresses AS (
    SELECT oversample('streets', 'id') as street_id FROM range(per_county(30000)) ORDER BY street_id
);

CREATE OR REPLACE TABLE addresses AS (
    SELECT 
        nextval('address_seq') as id,
        street_id, 
        row_number() OVER (PARTITION BY street_id) as street_number 
    FROM tmp_addresses
);



,Count
0,15000000


In [7]:
CREATE OR REPLACE TABLE families AS (
    SELECT 
        id,
        id as address_id,
        faker_last_name() as family_name,
    from addresses);

,Count
0,15000000


In [8]:
CREATE OR REPLACE TABLE people AS (
    SELECT 
        range as id,
        oversample('families', 'id') as family_id,
        faker_first_name() as person_name,
        random_randint(1, 90) as person_age
    FROM range(per_county(100000))
);

,Count
0,50000000


In [2]:
COPY (SELECT * EXCLUDE (id, country_id, county_id, town_id, street_id, address_id, incode, outcode, prefix), concat(outcode, ' ', incode) as postcode FROM countries 
    JOIN counties ON countries.id = counties.country_id
    JOIN towns ON counties.id = towns.county_id
    JOIN streets ON towns.id = streets.town_id
    JOIN addresses ON streets.id = addresses.street_id
    JOIN families ON addresses.id = families.address_id
    JOIN people on families.id = people.family_id
    ORDER BY country, county_name, town_name, street_name, street_number, family_name, person_name) TO 'synthetic_people.csv' (FORMAT CSV, PARTITION_BY (country), OVERWRITE_OR_IGNORE 1);

,Count
0,0


,count(country_id)
0,29
1,19
2,28
3,27
